# Exercices 6

# Exercise : Contamination dans une rivière

Le but de cet exercice est d'implémenter une équation d'advection-diffusion-réaction en 1D dans un cas réaliste.

La digue du bassin de résidus d'une mine d'or s'écroule et relâche de l'eau contaminée au cyanure dans une rivière. (Ce scénario est inspiré par la [catastrophe de 2000 de Baia Mare en Roumanie](https://en.wikipedia.org/wiki/2000_Baia_Mare_cyanide_spill). L'eau toxique va alors être transportée dans la rivière (par advection) à la même vitesse que son débit. Durant ce transport, le polluant va aussi se diffuser et se dégrader dans le cours d'eau. L'évolution de la contamination peut être décrite avec une équation de diffusion-advection-réaction :

$$\frac{\partial C}{\partial t} = - \frac{\partial Q}{\partial x} - v \frac{\partial C}{\partial x}\ - \gamma C,$$

avec 

$$Q = - D\frac{\partial C}{\partial x}.$$

1) Comme condition initiale, distribuez l'eau polluée entre les distances de 400 et 600 m et supposez que cette eau n'est pas diluée dans la rivière (`Cinit = 1000`). Utilisez une condition limite en aval de flux nul, et une condition limite fixe en amont pour forcer l'arrivée d'eau non contaminée. Implémentez le modèle dans un bloc de code ci-dessous en vous référant au tableau ci-dessous pour les paramètres. Profitez-en pour essayer les deux types de discrétisation ("upwind" et "downwind") pour l'advection, qui diffèrent par un simple décalage des indices. Quelle méthode fonctionne et quelle méthode ne fonctionne pas ?

2) Dans un deuxième temps, nous allons implémenter le même modèle en inversant l'axe $x$ de façon symétrique : i) la fuite est localisée sur le côté droit, ii) le signe de la vitesse est inversé, iii) les conditions aux bords sont inversées. Nous utiliserons le même code qui fonctionnera avec le paramètre `Q=1` dans le cas original et `Q=2` dans le cas inversé. Il s'agira donc d'introduire le paramètre `Q` ainsi que des clauses conditionnelles dans le code spécifiques à chaque cas (`Q=1` pour l'original et `Q=2` pour l'inverse).

### Paramètres pour le problème de la rivière

| **Paramètres**                             | **Valeurs**      |
|--------------------------------------------|------------------|
| Longueur du domaine modélisé               | 10 km            |
| Concentration initiale du cyanure          | 1000 ppm         |
| Coordonnées de la fuite (Q=1)              | 400 à 600 m      |
| Coordonnées de la fuite (Q=2)              | 9400 à 9600 m    |
| Diffusivité du cyanure                     | 5 m²/s           |
| Constante de décroissance $\gamma$         | 0.0001           |
| Vitesse de la rivière (Q=1)                | 2 m/s            |
| Vitesse de la rivière (Q=2)                | -2 m/s           |
| Nombre de cellules                         | 500              |
| Temps total                                | 1.5 hrs          |
 
 
![](./fig/cyanide.png)
*Représentation de l'advection et diffusion d'une fuite de cyanure dans une rivière*
 
### ✅ **À vous de faire !** 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt  
from IPython.display import display, clear_output

Q=2

# Physical parameters
D = 5.0  # diffusivity m^2/s
Lx = 10000.0  # length of model, m

C0 = 1000.0  # initial concentration ppm
if Q==1:
    leak = [400, 600]  # coordinate of leaks (start, end)
    vx = 2.0  # current velocity m/s
else:
    leak = [9400, 9600]  # coordinate of leaks (start, end)
    vx = -2.0  # current velocity m/s
    
ttot = 1.5 * 60 * 60  # total time in seconds
gamma = 1e-4  # rate of decay /s

# Numerical parameters
nx = 500  # number of cells
dx = Lx / (nx - 1)  # size of cell
x = np.linspace(0, Lx, nx)  # x-coordinates
dt = min(dx**2 / (2.1 * D), dx / (10 * np.abs(vx)))  # timestep
nt = int(ttot / dt)  # number of timestep
nout = round(nt / 30)  # plot modulator

# Initial conditions
C = np.zeros(nx)  # concentration array
C[int(leak[0] / dx):int(leak[1] / dx) + 1] = C0  # set concentration at leak

# Plot initial
fig, ax = plt.subplots() # create figure and axis

# Action
for it in range(nt):
    # Compute diffusion
    qx = - D * (C[1:] - C[:-1]) / dx  # flux
    dCdt_d = - (qx[1:] - qx[:-1]) / dx  # change in concentration /t
    C[1:-1] += dt * dCdt_d  # update rule

    # Compute advection
    dCdt_a = -vx * (C[1:] - C[:-1]) / dx  # change due to advection
    if Q==1:
        C[1:] += dt * dCdt_a    # update rule
    else:
        C[:-1] += dt * dCdt_a

    # REACTION
    dCdt_r = - C * gamma
    C     += dt * dCdt_r   # mise a jour de la reaction

    # Boundary conditions
    if Q==1:
        C[0] = 0       # left bc
        C[-1] = C[-2]  # right bc
    else:        
        C[0] = C[1] # left bc
        C[-1] = 0   # right bc

    # Plot
    if it % nout == 0:
        clear_output(wait=True)  # Clear the output in VS Code
        ax.cla() 
        ax.plot(x / 1000, C, 'k')
        ax.set_xlabel('horizontal distance (km)')
        ax.set_ylabel('concentration (ppm)')
        ax.set_title('concentration au temps '+str(int(dt*it/60))+ ' minutes')
        plt.axis([0, Lx / 1000, 0, C0 + 150])
        display(fig)
 